# Evolutionary games in Python

In [2]:
import numpy as np

## Playing a fixed number of games

In [103]:
def play_round(game, p0, p1, actions_hist):
    """
    Plays one round of a game whose payoff matrix is `game`,
    with two players whose strategies are the functions p1 and p2,
    with a past `history` of the players' actions, which are elements
    of {0, ..., n-1}, n being the number of rows of `game`.

    Payoff matrix `game`:
    rows: player actions
    columns: opponent actions
    actions: 0 = cooperate, 1 = defect
    """
    # each player's action
    p0_action = p0(actions_hist, player=0)
    p1_action = p1(actions_hist, player=1)

    # each player's payoff
    p0_payoff = game[p0_action, p1_action]
    p1_payoff = game[p1_action, p0_action]
    
    return (p0_action, p1_action), (p0_payoff, p1_payoff)


def play(game, p0, p1, actions_hist=[], n=1):
    """
    Play `n` rounds of `game` with players `p0` and `p1` starting
    from `init_actions`.
    """
    if len(actions_hist) == 0:
        payoffs_hist = []
    else:
        init_actions = actions_hist[0]
        payoffs_hist = [(game[init_actions], game[tuple(reversed(init_actions))])]
    
    for i in range(1, n):
        a, p = play_round(game, p0, p1, actions_hist)
        payoffs_hist.append(p)
        actions_hist.append(a)
        
    return actions_hist, payoffs_hist

### Some strategies

In [101]:
def always_defect(actions_hist=[], player=0):
    """
    The strategy that always defects no matter what.
    """
    return 1

def always_cooperate(actions_hist=[], player=0):
    """
    The strategy that always defects no matter what.
    """
    return 0

def tit_for_tat(actions_hist=[], player=0):
    """
    Plays the opponent's last move.
    """
    if len(actions_hist) == 0:
        return 0
        
    last_play = actions_hist[-1]
    return last_play[not player]

def tit_for_two_tats(actions_hist=[], player=0):
    """
    Defect if the opponent defects twice in a row,
    otherwise cooperate.
    """
    if len(actions_hist) == 0:
        return 0
        
    last_play = actions_hist[-1]
    second_to_last_play = actions_hist[-2]
    
    return last_play[not player] and second_to_last_play[not player]

def grudger(actions_hist=[], player=0):
    """
    Cooperates until the first defection of the opponent,
    then defects forever.
    """
    if len(actions_hist) == 0:
        return 0
        
    nplays = len(actions_hist)
    opponent_hist = [actions_hist[i][not player] for i in range(nplays)]
    
    return int(1 in opponent_hist)

def random(actions_hist=[], player=0):
    """
    Fair coin flip.
    """
    return np.random.default_rng().choice([0,1])

### Testing it out

In [104]:
# a payoff matrix for the prisoner's dilemma
prisoners_dilemma = np.array([[2, 0],
                              [3, 1]])

# both players cooperate initially
init_actions = [(0, 0)]

actions, payoffs = play(
    game = prisoners_dilemma,
    p0 = tit_for_tat,
    p1 = random,
    n = 10
)
print('actions (p0, p1): ', actions)
print('payoffs (p0, p1): ', payoffs)

actions (p0, p1):  [(0, 0), (0, 0), (0, 1), (1, 0), (0, 1), (1, 1), (1, 0), (0, 0), (0, 1)]
payoffs (p0, p1):  [(2, 2), (2, 2), (0, 3), (3, 0), (0, 3), (1, 1), (3, 0), (2, 2), (0, 3)]


## Tournaments

In [126]:
def round_robin(game, players, nrounds=1):
    """
    A round-robin tournament (all-to-all) between a list of players.
    """
    nplayers = len(players)
    payoffs = np.empty((nrounds, nplayers, nplayers))
    actions = np.empty((nrounds, nplayers, nplayers))

    for key0, p0 in enumerate(players):
        for key1, p1 in enumerate(players):
            if key0 > key1:
                a, p = play(game, p0, p1, n=nrounds)
                payoffs[:, key0, key1] = np.array(p)[:, 0]
                payoffs[:, key1, key0] = np.array(p)[:, 1]

                print(np.array(p))
                actions[:, key0, key1] = np.array(a)[:, 0]
                actions[:, key1, key0] = np.array(a)[:, 1]

    return acrions, payoffs


In [130]:
a, p = play(prisoners_dilemma, tit_for_tat, tit_for_tat, n=1)
print(a)
print(p)

[(0, 0), (0, 0), (0, 1), (1, 0), (0, 1), (1, 1), (1, 0), (0, 0), (0, 1)]
[(2, 2)]


In [127]:
# list of contestants
players = [tit_for_tat, tit_for_two_tats, random, grudger, always_cooperate, always_defect]

round_robin(prisoners_dilemma, players, 1)

[[2 2]]


ValueError: could not broadcast input array from shape (9,) into shape (1,)